# F1 Turbulence Simulation - Experiment Runner

This notebook provides an interface for running F1 aerodynamics simulations with:
- Single experiment runs with custom parameters
- Automated parameter sweeps (Reynolds number, ride height)
- Result visualization and analysis

## Quick Start
1. Run the **Setup** section once
2. Choose either **Single Experiment** or **Parameter Sweep**
3. Analyze results in the **Analysis** section

---
## Setup
Run this cell first to import all necessary modules.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Import simulation modules
from config import SimulationConfig, create_parameter_sweep, QUICK_TEST, STANDARD_RUN
from runner import run_simulation, run_and_save, SimulationResults
from parameter_sweep import run_parameter_sweep, ParameterSweep

print("✓ All modules loaded successfully")
print("\nReady to run experiments!")

✓ All modules loaded successfully

Ready to run experiments!


---
## Option 1: Single Experiment

Run a single simulation with your chosen parameters.

In [2]:
# Configure your experiment
config = SimulationConfig(
    # Domain size
    nx=400,
    ny=100,
    
    # Physics parameters - MODIFY THESE
    reynolds=10000,        # ← Reynolds number
    u_inlet=0.1,           # Inlet velocity
    
    # Geometry - MODIFY THESE
    ride_height=5,         # ← Distance from ground to wing base
    wing_x_pos=50,
    wing_length=30,
    wing_slope=0.5,
    wing_type="triangle",  # Options: "triangle" or "reverse_triangle"
    
    # Ground configuration
    ground_type="no_slip", # Options: "no_slip" or "slip"
    
    # Simulation control
    total_steps=10000,
    monitor_interval=100,
    dashboard_interval=500,
    
    # Output
    save_flow_field=True,
    save_force_history=True,
    output_dir="results/single_runs",
    
    # Experiment name
    run_id="my_experiment"  # Optional: give your run a name
)

print(f"Configuration: {config.get_run_name()}")
print(f"  Reynolds: {config.reynolds}")
print(f"  Ride Height: {config.ride_height}")
print(f"  Total Steps: {config.total_steps}")

Configuration: my_experiment
  Reynolds: 10000
  Ride Height: 5
  Total Steps: 10000


In [3]:
# Run the simulation
output_path = run_and_save(config, verbose=True)

print(f"\n{'='*60}")
print(f"Results saved to: {output_path}")
print(f"{'='*60}")


Starting Simulation: my_experiment
Reynolds: 10000, Ride Height: 5
Domain: 400x100, Steps: 10000

   [AERO] Drag: -7.5067 | Lift: 0.4333
   --> LIFT GENERATED (Car is flying!)
Step 0/10000
   [AERO] Drag: -6.7814 | Lift: -0.3876
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 100/10000
   [AERO] Drag: -6.5717 | Lift: -0.0571
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 200/10000
   [AERO] Drag: -6.5576 | Lift: -0.0165
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 300/10000
   [AERO] Drag: -6.5522 | Lift: -0.1846
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 400/10000
   [AERO] Drag: -6.4962 | Lift: -0.5045
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 500/10000
   [AERO] Drag: -6.4387 | Lift: -0.2034
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 600/10000
   [AERO] Drag: -6.4105 | Lift: -0.3643
   --> DOWNFORCE GENERATED (Ground Effect Active)
Step 700/10000
   [AERO] Drag: -6.4518 | Lift: -0.1097
   --> DOWNFORCE GENERATED (Ground 

---
## Option 2: Quick Test

Run a quick test with reduced resolution and steps to verify everything works.

In [ ]:
# Quick test configuration (fast execution)
quick_config = SimulationConfig(
    nx=200,              # Reduced resolution
    ny=50,
    reynolds=5000,
    ride_height=5,
    total_steps=1000,    # Only 1000 steps
    monitor_interval=100,
    output_dir="results/quick_test",
    run_id="quick_test"
)

output_path = run_and_save(quick_config, verbose=True)

---
## Option 3: Parameter Sweep - Reynolds Number

Sweep over different Reynolds numbers at fixed ride height.

In [ ]:
# Define Reynolds numbers to test
reynolds_values = [5000, 10000, 15000, 20000]

# Fixed ride height
ride_height_values = [5]

# Base configuration (shared parameters)
base_config = SimulationConfig(
    nx=400,
    ny=100,
    total_steps=10000,
    monitor_interval=100,
    save_dashboard=False,
    experiment_name="reynolds_sweep"
)

print(f"Reynolds Sweep Configuration:")
print(f"  Reynolds values: {reynolds_values}")
print(f"  Fixed ride height: {ride_height_values[0]}")
print(f"  Total simulations: {len(reynolds_values)}")
print(f"\nThis will take approximately {len(reynolds_values) * 5} minutes...\n")

In [ ]:
# Run the Reynolds sweep
reynolds_sweep = run_parameter_sweep(
    reynolds_values=reynolds_values,
    ride_height_values=ride_height_values,
    base_config=base_config,
    output_dir="results/reynolds_sweep",
    verbose=True
)

print("\n✓ Reynolds sweep complete!")

---
## Option 4: Parameter Sweep - Ride Height

Sweep over different ride heights at fixed Reynolds number.

In [ ]:
# Fixed Reynolds number
reynolds_values = [10000]

# Define ride heights to test
ride_height_values = [3, 5, 7, 10, 15]

# Base configuration
base_config = SimulationConfig(
    nx=400,
    ny=100,
    total_steps=10000,
    monitor_interval=100,
    save_dashboard=False,
    experiment_name="height_sweep"
)

print(f"Ride Height Sweep Configuration:")
print(f"  Fixed Reynolds: {reynolds_values[0]}")
print(f"  Ride height values: {ride_height_values}")
print(f"  Total simulations: {len(ride_height_values)}")
print(f"\nThis will take approximately {len(ride_height_values) * 5} minutes...\n")

In [ ]:
# Run the ride height sweep
height_sweep = run_parameter_sweep(
    reynolds_values=reynolds_values,
    ride_height_values=ride_height_values,
    base_config=base_config,
    output_dir="results/height_sweep",
    verbose=True
)

print("\n✓ Ride height sweep complete!")

---
## Option 5: Full Parameter Sweep (Re × Height)

Sweep over BOTH Reynolds numbers AND ride heights (full factorial design).

In [ ]:
# Define parameter ranges
reynolds_values = [5000, 10000, 15000]
ride_height_values = [3, 5, 7, 10]

# Calculate total simulations
total_sims = len(reynolds_values) * len(ride_height_values)

print(f"Full Parameter Sweep Configuration:")
print(f"  Reynolds values: {reynolds_values}")
print(f"  Ride height values: {ride_height_values}")
print(f"  Total simulations: {len(reynolds_values)} × {len(ride_height_values)} = {total_sims}")
print(f"\n⚠️  This will take approximately {total_sims * 5} minutes...\n")

# Base configuration
base_config = SimulationConfig(
    nx=400,
    ny=100,
    total_steps=10000,
    monitor_interval=100,
    save_dashboard=False,
    experiment_name="full_sweep"
)

In [ ]:
# Run the full parameter sweep
full_sweep = run_parameter_sweep(
    reynolds_values=reynolds_values,
    ride_height_values=ride_height_values,
    base_config=base_config,
    output_dir="results/full_sweep",
    verbose=True
)

print("\n✓ Full parameter sweep complete!")

---
## Option 6: Quick Test Sweep

Run a quick parameter sweep with reduced resolution to test the workflow.

In [ ]:
# Quick test sweep
quick_sweep = run_parameter_sweep(
    reynolds_values=[5000, 10000],
    ride_height_values=[5, 10],
    base_config=SimulationConfig(
        nx=200,
        ny=50,
        total_steps=1000,
        monitor_interval=100,
        experiment_name="quick_test_sweep"
    ),
    output_dir="results/quick_test_sweep",
    verbose=True
)

print("\n✓ Quick test sweep complete!")

---
## Results Analysis

Load and analyze results from parameter sweeps.

### Load Sweep Results

In [ ]:
# Load summary data from a sweep
sweep_dir = "results/full_sweep"  # ← Change this to your sweep directory

summary_file = Path(sweep_dir) / "sweep_summary.csv"

if summary_file.exists():
    df = pd.read_csv(summary_file)
    print(f"Loaded {len(df)} results from {sweep_dir}")
    print("\nSummary Statistics:")
    display(df.describe())
    print("\nFirst few results:")
    display(df.head())
else:
    print(f"No results found at {summary_file}")
    print("Run a parameter sweep first!")

### Custom Analysis Plots

In [ ]:
# Plot 1: Drag vs Reynolds Number
if 'df' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Group by ride height if available
    if df['ride_height'].nunique() > 1:
        for height in sorted(df['ride_height'].unique()):
            subset = df[df['ride_height'] == height]
            axes[0].plot(subset['reynolds'], subset['mean_drag'], 
                        marker='o', label=f'Height {height}')
            axes[1].plot(subset['reynolds'], subset['mean_lift'], 
                        marker='s', label=f'Height {height}')
    else:
        axes[0].plot(df['reynolds'], df['mean_drag'], marker='o')
        axes[1].plot(df['reynolds'], df['mean_lift'], marker='s')
    
    axes[0].set_xlabel('Reynolds Number')
    axes[0].set_ylabel('Mean Drag Force')
    axes[0].set_title('Drag vs Reynolds Number')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('Reynolds Number')
    axes[1].set_ylabel('Mean Lift Force')
    axes[1].set_title('Lift vs Reynolds Number (Negative = Downforce)')
    axes[1].axhline(0, color='black', linestyle='--', alpha=0.5)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot 2: Drag vs Ride Height
if 'df' in locals() and df['ride_height'].nunique() > 1:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Group by Reynolds number if available
    if df['reynolds'].nunique() > 1:
        for re in sorted(df['reynolds'].unique()):
            subset = df[df['reynolds'] == re]
            axes[0].plot(subset['ride_height'], subset['mean_drag'], 
                        marker='o', label=f'Re {int(re)}')
            axes[1].plot(subset['ride_height'], subset['mean_lift'], 
                        marker='s', label=f'Re {int(re)}')
    else:
        axes[0].plot(df['ride_height'], df['mean_drag'], marker='o')
        axes[1].plot(df['ride_height'], df['mean_lift'], marker='s')
    
    axes[0].set_xlabel('Ride Height')
    axes[0].set_ylabel('Mean Drag Force')
    axes[0].set_title('Drag vs Ride Height')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('Ride Height')
    axes[1].set_ylabel('Mean Lift Force')
    axes[1].set_title('Lift vs Ride Height (Negative = Downforce)')
    axes[1].axhline(0, color='black', linestyle='--', alpha=0.5)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot 3: Downforce Ratio Analysis
if 'df' in locals():
    plt.figure(figsize=(10, 6))
    
    scatter = plt.scatter(
        df['mean_drag'], 
        df['mean_lift'],
        c=df['downforce_ratio'],
        s=150,
        cmap='RdYlGn',
        alpha=0.7,
        edgecolors='black',
        linewidth=1.5
    )
    
    plt.axhline(0, color='black', linestyle='--', alpha=0.3)
    plt.xlabel('Mean Drag Force', fontsize=12)
    plt.ylabel('Mean Lift Force', fontsize=12)
    plt.title('Drag-Lift Trade-off Across Parameter Space', fontsize=14)
    cbar = plt.colorbar(scatter, label='Downforce Ratio')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nOptimal configurations (highest downforce ratio):")
    top_configs = df.nlargest(3, 'downforce_ratio')[['run_name', 'reynolds', 'ride_height', 'downforce_ratio', 'mean_lift']]
    display(top_configs)

### Load Individual Run for Detailed Analysis

In [ ]:
# Load detailed force history from a specific run
run_dir = "results/full_sweep/Re10000_h5"  # ← Change to your run directory

force_file = Path(run_dir) / "force_history.npz"

if force_file.exists():
    data = np.load(force_file)
    steps = data['steps']
    drag = data['drag']
    lift = data['lift']
    
    # Plot time evolution
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    axes[0].plot(steps, drag, 'r-', linewidth=1.5)
    axes[0].set_ylabel('Drag Force', fontsize=12)
    axes[0].set_title(f'Force Evolution - {run_dir}', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(steps, lift, 'b-', linewidth=1.5)
    axes[1].axhline(0, color='black', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Simulation Step', fontsize=12)
    axes[1].set_ylabel('Lift Force (Negative = Downforce)', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display statistics
    steady_start = len(drag) // 5
    print("\nSteady-State Statistics (excluding initial transient):")
    print(f"  Drag:  {np.mean(drag[steady_start:]):.4f} ± {np.std(drag[steady_start:]):.4f}")
    print(f"  Lift:  {np.mean(lift[steady_start:]):.4f} ± {np.std(lift[steady_start:]):.4f}")
else:
    print(f"No force history found at {force_file}")

---
## Export Results

Export filtered results to CSV for further analysis.

In [ ]:
# Example: Export only configurations with significant downforce
if 'df' in locals():
    downforce_configs = df[df['downforce_ratio'] > 0.8]
    
    output_file = "analysis/downforce_configurations.csv"
    Path(output_file).parent.mkdir(exist_ok=True)
    downforce_configs.to_csv(output_file, index=False)
    
    print(f"Exported {len(downforce_configs)} configurations with >80% downforce")
    print(f"Saved to: {output_file}")
    display(downforce_configs)

---
## Notes

### Parameter Guidelines:
- **Reynolds Number**: Controls turbulence intensity
  - 5,000-10,000: Transitional flow
  - 10,000-20,000: Turbulent flow
  - Higher Re = More computational cost

- **Ride Height**: Distance from ground to wing base
  - Lower height → Stronger ground effect
  - Typically: 3-15 grid units

### Computational Cost:
- Standard run (400×100, 10k steps): ~5 minutes
- High-res run (800×200, 15k steps): ~30 minutes
- Full sweep (12 configs): ~60 minutes

### Output Structure:
```
results/
├── single_runs/
│   └── Re10000_h5/
│       ├── config.json
│       ├── force_history.npz
│       ├── force_history.png
│       ├── final_flow.png
│       └── summary_stats.json
└── full_sweep/
    ├── sweep_metadata.json
    ├── sweep_summary.csv
    ├── parameter_heatmap.png
    └── Re10000_h5/...
```